# Using ModelingToolkit

In [ ]:
using ModelingToolkit 
using MethodOfLines 
using OrdinaryDiffEq 
using DomainSets
using FFTW 
using Plots

## Section 1: Introduction 
More later. 

## Section 2: 1D Heat Equation 
Solves the heat equation in 1D space ($x$) and time ($t$).  

In [ ]:
# Parameters, variables, and derivatives
@parameters x t
@variables u(..)
Dxx = Differential(x)^2
Dt = Differential(t)

#PDE
alpha = 1 # thermal diffusivity  
eq  = [ Dt(u(x,t)) ~ alpha*Dxx(u(x,t)) ] 

mypi = 3.1415
u0(x,t) = sin(mypi*x)*exp(-pi*c*t)

# Initial and boundary conditions
# Initial velocity is still missing 
bcs = [ u(0.,t) ~ 0.,# for all t > 0
        u(1.,t) ~ 0.,
        u(x,0) ~ sin(mypi*x)]

# Space and time domains
maxt = .5 
domains = [x ∈ (0.0,1.0), 
           t ∈ (0.0,maxt)]

@named pdesys = PDESystem(eq,bcs,domains,[x,t],[u(x,t)])

#  Define discretization 
N = 32
order = 2 
discretization = MOLFiniteDifference([x=>N], t, approx_order=order)

# Convert the PDE problem into an ODE problem
println("Discretization:")
prob = discretize(pdesys,discretization)

println("Solve:")
sol = solve(prob, TRBDF2(),saveat=maxt/10)

# Plot results
surface(sol[t], sol[x], sol[u(x,t)],xlabel="t",ylabel="x") 

## Section 3: 1D Wave Equation - Case (1/4): Linear Undamped Unforced 
Solves the wave equation in 1D space ($x$) and time ($t$) separated into two first order in time equations. More.     

### Section 1.3: Case (1/4): Time Integration 

In [ ]:
# Parameters, variables, and derivatives
@parameters x t
@variables u(..) v(..)
Dxx = Differential(x)^2
Dtt = Differential(t)^2
Dt = Differential(t)
Dx = Differential(x)

#2D PDE
c=1 # wave velocity 
eq  = [ Dt(u(x,t)) ~ v(x,t),
        Dt(v(x,t)) ~ c^2*Dxx(u(x,t))] 

# Avoid issue with pi in ModelingToolkit
# see e.g. https://discourse.julialang.org/t/discretization-of-semi-coupled-wave-and-diffusion-equations-fails-for-step-size-of-pi-25n-using-methodoflines-jl/106091 
mypi = 3.1415

# Initial and boundary conditions
# Initial velocity is still missing 
bcs = [ u(0.,t) ~ 0.,         # left BC for position u
        u(1.,t) ~ 0.,         # right BC for position u 
        v(0,t) ~ 0.,          # left BC for velocity v 
        v(1,t) ~ 0.,          # right BC for velocity v
        u(x,0) ~ sin(mypi*x), # IC for position u 
        v(x,0) ~ 0.0]         # IC for velocity v 

# Space and time domains
maxt = 10
domains = [x ∈ (0.0,1.0), 
           t ∈ (0.0,maxt)]

@named pdesys = PDESystem(eq,bcs,domains,[x,t],[u(x,t), v(x,t)])

#  Define discretization 
N = 32
order = 2 
discretization = MOLFiniteDifference([x=>N], t, approx_order=order)

# Convert the PDE problem into an ODE problem
println("Discretization:")
prob = discretize(pdesys,discretization)

println("Solve:")
sol = solve(prob, TRBDF2(), reltol = 1e-4, abstol = 1e-8)

# Plot results
p1 = surface(sol[t], sol[x], sol[u(x,t)],xlabel="t",ylabel="x",title="position") 
p2 = surface(sol[t], sol[x], sol[v(x,t)],xlabel="t",ylabel="x",title="velocity") 
plot(p1,p2,layout=(1,2))

### Section 2.3: Case (1/4): Plot Time Traces 

In [ ]:
xquart = div(N,4); xmid = div(N,2); x3quart = 3*xquart
uu = sol[u(x,t)]; vv = sol[v(x,t)]; 
plot(sol[t],uu[xquart,:],xlabel="t",label="x=.25")
plot!(sol[t],uu[xmid,:],xlabel="t",label="x=.5")
p1 = plot!(sol[t],uu[3*xquart,:],xlabel="t",label="x=.75",title="position")
plot(sol[t],vv[xquart,:],xlabel="t",label="x=.25")
plot!(sol[t],vv[xmid,:],xlabel="t",label="x=.5")
p2 = plot!(sol[t],vv[3*xquart,:],xlabel="t",label="x=.75",title="velocity")
plot(p1,p2,layout=(1,2)) 

## Section 4: 1D Wave Equation -  Case (2/4): Linear Damped Unforced 
Damped version of above. 

### Section 1.4: Case (2/4): Time Integration 

In [ ]:
# Parameters, variables, and derivatives
@parameters x t
@variables u(..) v(..)
Dxx = Differential(x)^2
Dtt = Differential(t)^2
Dt = Differential(t)
Dx = Differential(x)

#2D PDE
c=1 # wave velocity 
ga=.5 # damping coefficient  
eq  = [ Dt(u(x,t)) ~ v(x,t),
        Dt(v(x,t)) ~ c^2*Dxx(u(x,t)) - ga*v(x,t)] 

# Avoid issue with pi in ModelingToolkit
mypi = 3.1415

# Initial and boundary conditions
# Initial velocity is still missing 
bcs = [ u(0.,t) ~ 0.,         # left BC for position u
        u(1.,t) ~ 0.,         # right BC for position u 
        v(0,t) ~ 0.,          # left BC for velocity v 
        v(1,t) ~ 0.,          # right BC for velocity v
        u(x,0) ~ sin(mypi*x), # IC for position u 
        v(x,0) ~ 0.0]         # IC for velocity v 

# Space and time domains
maxt = 10
domains = [x ∈ (0.0,1.0), 
           t ∈ (0.0,maxt)]

@named pdesys = PDESystem(eq,bcs,domains,[x,t],[u(x,t), v(x,t)])

#  Define discretization 
N = 32
order = 2 
discretization = MOLFiniteDifference([x=>N], t, approx_order=order)

# Convert the PDE problem into an ODE problem
println("Discretization:")
prob = discretize(pdesys,discretization)

println("Solve:")
sol = solve(prob, TRBDF2(), reltol = 1e-4, abstol = 1e-8)

# Plot results
p1 = surface(sol[t], sol[x], sol[u(x,t)],xlabel="t",ylabel="x",title="position") 
p2 = surface(sol[t], sol[x], sol[v(x,t)],xlabel="t",ylabel="x",title="velocity") 
plot(p1,p2,layout=(1,2))

### Section 2.4: Case (2/4): Plot Time Traces 

In [ ]:
# plot traces 
xquart = div(N,4); xmid = div(N,2); x3quart = 3*xquart
uu = sol[u(x,t)]; vv = sol[v(x,t)]; 
plot(sol[t],uu[xquart,:],xlabel="t",label="x=.25")
plot!(sol[t],uu[xmid,:],xlabel="t",label="x=.5")
p1 = plot!(sol[t],uu[3*xquart,:],xlabel="t",label="x=.75",title="position")
plot(sol[t],vv[xquart,:],xlabel="t",label="x=.25")
plot!(sol[t],vv[xmid,:],xlabel="t",label="x=.5")
p2 = plot!(sol[t],vv[3*xquart,:],xlabel="t",label="x=.75",title="velocity")
plot(p1,p2,layout=(1,2)) 

## Section 5: 1D Wave Equation - Case (3/4): Linear Damped Forced 
Forced version of above. 
Harmonic balance method is exact after sufficientl long time. 
Establish link with Helmholtz equation equation  through the wave number. `

### Section 1.5: Case (3/4): Time Integration 

In [ ]:
# Parameters, variables, and derivatives
@parameters x t
@variables u(..) v(..)
Dxx = Differential(x)^2
Dtt = Differential(t)^2
Dt = Differential(t)
Dx = Differential(x)

# Avoid issue with pi in ModelingToolkit
mypi = 3.1415

#2D PDE
c=1      # wave velocity 
ga=5    # damping coefficient 
omd=5    # angular frequency of driving force - to write: write this as 2*pi*freqd 
F0=1000     # amplitude of angular force  
eq  = [ Dt(u(x,t)) ~ v(x,t),
        Dt(v(x,t)) ~ c^2*Dxx(u(x,t)) - ga*v(x,t) + F0*sin(3*mypi*x)*sin(omd*t)] 

# Initial and boundary conditions
# Initial velocity is still missing 
bcs = [ u(0.,t) ~ 0.,         # left BC for position u
        u(1.,t) ~ 0.,         # right BC for position u 
        v(0,t) ~ 0.,          # left BC for velocity v 
        v(1,t) ~ 0.,          # right BC for velocity v
        u(x,0) ~ sin(mypi*x), # IC for position u 
        v(x,0) ~ 0.0]         # IC for velocity v 

# Space and time domains
maxt = 40
domains = [x ∈ (0.0,1.0), 
           t ∈ (0.0,maxt)]

@named pdesys = PDESystem(eq,bcs,domains,[x,t],[u(x,t), v(x,t)])

#  Define discretization 
N = 100 # 32
order = 2 
discretization = MOLFiniteDifference([x=>N], t, approx_order=order)

# Convert the PDE problem into an ODE problem
println("Discretization:")
prob = discretize(pdesys,discretization)

println("Solve:")
sol = solve(prob, TRBDF2(), reltol = 1e-4, abstol = 1e-8)

# Plot results
p1 = surface(sol[t], sol[x], sol[u(x,t)],xlabel="t",ylabel="x",title="position") 
p2 = surface(sol[t], sol[x], sol[v(x,t)],xlabel="t",ylabel="x",title="velocity") 
plot(p1,p2,layout=(1,2))

### Section 2.5: Case (3/4): Plot Time Traces 

In [ ]:
# plot traces 
xquart = div(N,4); xmid = div(N,2); x3quart = 3*xquart
uu = sol[u(x,t)]; vv = sol[v(x,t)]; 
plot(sol[t],uu[xquart,:],xlabel="t",label="x=.25")
plot!(sol[t],uu[xmid,:],xlabel="t",label="x=.5")
p1 = plot!(sol[t],uu[x3quart,:],xlabel="t",label="x=.75",title="position")
plot(sol[t],vv[xquart,:],xlabel="t",label="x=.25")
plot!(sol[t],vv[xmid,:],xlabel="t",label="x=.5")
p2 = plot!(sol[t],vv[3*xquart,:],xlabel="t",label="x=.75",title="velocity")
plot(p1,p2,layout=(1,2)) 

### Section 3.5: Case (3/4): Time Step Information  

In [ ]:
p1 = plot(sol[t],xlabel="time steps",ylabel="t",label="t") 
p2 = bar(sol[t][2:end] - sol[t][1:end-1],xlabel="time steps",ylabel="dt",label="dt")
plot(p1,p2,layout=(1,2))

In [ ]:
ttrunc=300 
uutrunc = uu[:,ttrunc:end]
contour(uutrunc)

### Section 4.5: Case (3/4): Plot of Steady State  

### Section 5.5: Case (3/4): FFT of Steady State

In [ ]:
# definite time samples 
xquart = div(N,4); xmid = div(N,2); x3quart = 3*xquart
up1 = uutrunc[xquart,:]
up2 = uutrunc[xmid,:]
up3 = uutrunc[x3quart,:]

# perform FFT of sampled position data 
ufp1 = fft(up1)
ufp2 = fft(up2)
ufp3 = fft(up3)

#..set frequency axis 
dt = 0.1
Nt = length(up1) 
fmax = 1/(2.0*dt)
fstep = 2*fmax/Nt
fvec = Vector(0:fstep:fmax)

#..plot absolute value of FFT samples  
p1 = plot(fvec, 2.0/Nt * abs.(ufp1[1:length(fvec)]),label="x=.25 norm")
p2 = plot(fvec, angle.(ufp1[1:length(fvec)]),label="x=.25 phase")
p3 = plot(fvec, 2.0/Nt * abs.(ufp2[1:length(fvec)]),label="x=.5 norm")
p4 = plot(fvec, angle.(ufp2[1:length(fvec)]),label="x=.5 phase")
p5 = plot(fvec, 2.0/Nt * abs.(ufp3[1:length(fvec)]),label="x=.75 norm")
p6 = plot(fvec, angle.(ufp3[1:length(fvec)]),label="x=.75 phase")
plot(p1, p2, p3, p4, p5, p6, layout = (3,2))

## Section 6: 1D Wave Equation -  Case (4/4): Non-linear 
Forced version of above. 

### Section 1.6: Case (4/4): Time Integration 

In [ ]:
# Parameters, variables, and derivatives
@parameters x t
@variables u(..) v(..)
Dxx = Differential(x)^2
Dtt = Differential(t)^2
Dt = Differential(t)
Dx = Differential(x)

#2D PDE
c=1     # wave velocity 
ga=5    # damping coefficient 
ga3=500 # non-linear damping coefficient 
omd=5   # angular frequency of driving force 
F0=1000   # amplitude of angular force  
eq  = [ Dt(u(x,t)) ~ v(x,t),
        Dt(v(x,t)) ~ c^2*Dxx(u(x,t)) - ga*v(x,t) - ga3*(v(x,t))^3 + F0*sin(3*mypi*x)*sin(omd*t)] 

# Avoid issue with pi in ModelingToolkit
mypi = 3.1415

# Initial and boundary conditions
# Initial velocity is still missing 
bcs = [ u(0.,t) ~ 0.,         # left BC for position u
        u(1.,t) ~ 0.,         # right BC for position u 
        v(0,t) ~ 0.,          # left BC for velocity v 
        v(1,t) ~ 0.,          # right BC for velocity v
        u(x,0) ~ sin(mypi*x), # IC for position u 
        v(x,0) ~ 0.0]         # IC for velocity v 

# Space and time domains
maxt = 1000
domains = [x ∈ (0.0,1.0), 
           t ∈ (0.0,maxt)]

@named pdesys = PDESystem(eq,bcs,domains,[x,t],[u(x,t), v(x,t)])

#  Define discretization 
N = 32
order = 2 
discretization = MOLFiniteDifference([x=>N], t, approx_order=order)

# Convert the PDE problem into an ODE problem
println("Discretization:")
prob = discretize(pdesys,discretization)

println("Solve:")
sol = solve(prob, TRBDF2(), reltol = 1e-4, abstol = 1e-8)

# Plot results
p1 = surface(sol[t], sol[x], sol[u(x,t)],xlabel="t",ylabel="x",title="position") 
p2 = surface(sol[t], sol[x], sol[v(x,t)],xlabel="t",ylabel="x",title="velocity") 
plot(p1,p2,layout=(1,2))

### Section 2.6: Case (4/4): Plot Time Traces 

In [ ]:
# plot traces 
xquart = div(N,4); xmid = div(N,2); x3quart = 3*xquart
uu = sol[u(x,t)]; vv = sol[v(x,t)]; 
plot(sol[t],uu[xquart,:],xlabel="t",label="x=.25")
plot!(sol[t],uu[xmid,:],xlabel="t",label="x=.5")
p1 = plot!(sol[t],uu[3*xquart,:],xlabel="t",label="x=.75",title="position")
plot(sol[t],vv[xquart,:],xlabel="t",label="x=.25")
plot!(sol[t],vv[xmid,:],xlabel="t",label="x=.5")
p2 = plot!(sol[t],vv[3*xquart,:],xlabel="t",label="x=.75",title="velocity")
plot(p1,p2,layout=(1,2)) 

### Section 3.6: Case (4/4): Time Step Information

In [ ]:
p1 = plot(sol[t],xlabel="time steps",ylabel="t",label="t") 
p2 = bar(sol[t][2:end] - sol[t][1:end-1],xlabel="time steps",ylabel="dt",label="dt")
plot(p1,p2,layout=(1,2))

### Section 4.6: Case (4/4): Steady State Solution 

In [ ]:
ttrunc=1000 
uutrunc = uu[:,ttrunc:end]
contour(uutrunc)

### Section 5.6: Case (4/4): FFT of Steady State Solution

In [ ]:
# definite time samples 
tstepstart=50 
up1 = uu[xquart,tstepstart:end]
up2 = uu[xmid,tstepstart:end]
up3 = uu[x3quart,tstepstart:end]

# perform FFT of sampled position data 
ufp1 = fft(up1)
ufp2 = fft(up2)
ufp3 = fft(up3)

#..set frequency axis 
dt = 0.1
Nt = length(up1)-tstepstart 
fmax = 1/(2.0*dt)
fstep = 2*fmax/Nt
fvec = Vector(0:fstep:fmax)

#..plot absolute value of FFT samples  
p1 = plot(fvec, 2.0/Nt * abs.(ufp1[1:length(fvec)]),label="x=.25 norm")
p2 = plot(fvec, angle.(ufp1[1:length(fvec)]),label="x=.25 phase")
p3 = plot(fvec, 2.0/Nt * abs.(ufp2[1:length(fvec)]),label="x=.5 norm")
p4 = plot(fvec, angle.(ufp2[1:length(fvec)]),label="x=.5 phase")
p5 = plot(fvec, 2.0/Nt * abs.(ufp3[1:length(fvec)]),label="x=.75 norm")
p6 = plot(fvec, angle.(ufp3[1:length(fvec)]),label="x=.75 phase")
plot(p1, p2, p3, p4, p5, p6, layout = (3,2))

In [ ]:
using NonlinearSolve

@parameters x y
@variables u(..)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

eq = Dxx(u(x, y)) + Dyy(u(x, y)) ~ 0

bcs = [u(0, y) ~ x * y,
       u(1, y) ~ x * y,
       u(x, 0) ~ x * y,
       u(x, 1) ~ x * y]


# Space and time domains
domains = [x ∈ Interval(0.0, 1.0),
           y ∈ Interval(0.0, 1.0)]

@named pdesys = PDESystem([eq], bcs, domains, [x, y], [u(x, y)])

dx = 0.1
dy = 0.1

# Note that we pass in `nothing` for the time variable `t` here since we
# are creating a stationary problem without a dependence on time, only space.
discretization = MOLFiniteDifference([x => dx, y => dy], nothing, approx_order=2)

prob = discretize(pdesys, discretization)
sol = NonlinearSolve.solve(prob, NewtonRaphson())

u_sol = sol[u(x, y)]

using Plots

heatmap(sol[x], sol[y], u_sol, xlabel="x values", ylabel="y values",
        title="Steady State Heat Equation")